In [1]:
from bs4 import BeautifulSoup
import configparser
import requests
import pandas as pd
import pickle
from constants_funcs import get_session, int_or_float_or_str
import json
with open('constants.json', 'r') as d:
    constants = json.load(d)

In [2]:
sess = get_session()
sess

using existing login session


In [3]:
search_term = '' # if you want to find specific players by name change this search term
all_id_url = constants['LLHEADER'] + '/backend-search.php?term=' + search_term
all_id_url

'https://www.learnedleague.com/backend-search.php?term='

In [4]:
ids_html = sess.get(all_id_url)
html_text = ids_html.text
soup = BeautifulSoup(html_text, 'html.parser')

In [5]:
rows = soup.find_all('a')
dic = dict() # CAVEAT: ONLY FINDS CURRENTLY ACTIVE PLAYERS
for row in rows:
    id = int(row['href'].split('?')[1])
    name = row.find('span').text
    loc = row.find('i').text
    dic[id] = {'name': name, 'loc': loc}

In [6]:
cur_id = 927
url = constants['LLHEADER'] + f'/profiles.php?{str(cur_id)}'
main_data = sess.get(f'{url}&9')
html_text = main_data.text
soup = BeautifulSoup(html_text, 'html.parser')
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html dir="ltr" lang="en-gb" xml:lang="en-gb" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<title>LL Profile: MateiroA</title>
<meta charset="utf-8">
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="/assets/css/main2.css" rel="stylesheet"/>
<link href="/images/logos/clownball_transparent.ico" rel="icon" type="image/ico"/>
<link href="/images/logos/clownball_transparent.ico" rel="shortcut icon" type="image/x-icon">
<meta content="The Greatest Trivia League in All the Land." name="description"/>
<meta content="trivia, league, learnedleague, thorsten, integrity, one-day, minileague" name="keywords"/>
<script type="text/javascript"><!--
	var spoiler_show = "[Reveal]";
	var spoiler_hide = "[Obscure]";
//--></script>
<!-- <script type="text/javascript" src="./styles/subs

In [7]:
def get_children(id):
    referrals_url = constants['LLHEADER'] + f'/profiles.php?{str(id)}&5'
    print(referrals_url)
    main_data = sess.get(referrals_url)
    html_text = main_data.text
    soup = BeautifulSoup(html_text, 'html.parser')
    header = soup.find(text="Direct Referrals ").next_element.next_element.next_element.next_element
    print(header)

In [3]:
class BasePlayer:
    def __init__(self, id: int):
        self.id: int = id

In [36]:
def build_base_player(id: int) -> dict:
    url = constants['LLHEADER'] + f'/profiles.php?{str(id)}'
    main_data = sess.get(url)
    html_text = main_data.text
    soup = BeautifulSoup(html_text, 'html.parser')
    ret = {'id': id}
    ret['deceased'] = False

    if 'This player is inactive.' in soup.text:
        ret['current_status'] = 'deactivated'
        ret['username'] = soup.find('div', attrs={'class': 'namediv'}).text.strip()
    else:
        box = soup.find('div', attrs={'class': 'topcont'})
        if '\t\tPassed away' in box.text:
            ret['current_status'] = 'deactivated'
            ret['deceased'] = True
        else:
            ret['current_status'] = box['class'][-1]
        branch_comp = soup.findAll('div', attrs={'class': 'demog_row'})[1]
        ret['branch_id'] = int(branch_comp.find('a')['href'].split('?')[-1])
        ret['branch_name'] = branch_comp.text.strip().split(': ')[1]
        genloc = soup.findAll('div', attrs={'class': 'demog_row'})[0].text.strip().split('\n\n')
        ret['college'] = (soup.find(text='College:').parent.parent.contents[2].strip()
            if soup.find(text='College:') else None)
        ret['gender'] = (soup.find(text='Gender:').parent.parent.text.split(':')[1].strip()
            if soup.find(text='Gender:') else None)
        ret['location'] = (soup.find(text='Location:').parent.parent.text.split(':')[1].strip()
            if soup.find(text='Location:') else None)
        ret['referrer'] = (int(soup.find(text='Referrer:').parent.parent.find('a')['href'].split('?')[-1])
            if soup.find(text='Referrer:') else 0)
        ret['name'] = soup.find('h1', attrs={'class': 'namecss'}).text
        ret['league'] = soup.find('div', attrs={'class': 'leaguelogodiv'}).text.strip()
    return ret
build_base_player(48408)

{'id': 48408,
 'deceased': False,
 'current_status': 'std',
 'branch_id': 20343,
 'branch_name': 'ZhangR',
 'college': None,
 'gender': None,
 'location': 'New York, New York',
 'referrer': 47621,
 'name': 'DattatreyaR',
 'league': 'Sugarloaf'}

In [37]:
with open('player_info.json', 'r') as d:
    players= json.load(d)
with open('player_profiles.json', 'r') as f:
    basic_players_profile = json.load(f)
for idx, player in enumerate(players.keys()):
    if player not in basic_players_profile.keys():
        try:
            basic_players_profile[player] = build_base_player(int(player))
        except:
            print(f'error on {player}')
    if idx % 1000 == 0:
        print(f'done {idx}')

done 0
done 1000
done 2000
done 3000
done 4000
done 5000
done 6000
done 7000
done 8000
done 9000
done 10000
done 11000
done 12000
error on 68032
done 13000
done 14000
done 15000
done 16000
done 17000
done 18000
done 19000
done 20000
done 21000
done 22000
done 23000
done 24000
done 25000
done 26000
done 27000


In [38]:
with open('player_profiles.json', 'w') as wp:
    json.dump(basic_players_profile, wp)
basic_players_profile

{'39204': {'id': 39204,
  'deceased': False,
  'current_status': 'std',
  'branch_id': 25924,
  'branch_name': 'ThompsonAM',
  'college': 'University of California, Santa Cruz',
  'gender': 'Female',
  'location': 'Berlin, Germany',
  'referrer': 9113,
  'name': 'AVeryMerryUnbirthday',
  'league': 'Skyline'},
 '35088': {'id': 35088,
  'deceased': False,
  'current_status': 'std',
  'branch_id': 11063,
  'branch_name': 'BradleyA',
  'college': 'Santa Clara University',
  'gender': 'Female',
  'location': 'Washington, DC',
  'referrer': 11064,
  'name': 'AagaardK',
  'league': 'Patagonia'},
 '65415': {'id': 65415,
  'deceased': False,
  'current_status': 'std',
  'branch_id': 22135,
  'branch_name': 'BrensingerB',
  'college': None,
  'gender': 'Male',
  'location': 'Chicago, Illinois',
  'referrer': 60483,
  'name': 'AakaluV',
  'league': 'Riviera'},
 '14777': {'id': 14777,
  'deceased': False,
  'current_status': 'pm',
  'branch_id': 10297,
  'branch_name': 'MellemK',
  'college': 'Uni

In [64]:
pp_df = pd.DataFrame.from_dict(basic_players_profile, orient='index')
pp_df.name.fillna(pp_df.username, inplace=True)
pp_df.username.fillna(pp_df.name, inplace=True)
pp_df.drop(labels=['name'], axis=1, inplace=True)
pp_df['branch_id'] = pp_df['branch_id'].astype('Int64')
pp_df['referrer'] = pp_df['referrer'].astype('Int64')
pp_df.to_csv('./player_profiles.csv', sep=',')

In [65]:
pp_df.dtypes

id                 int64
deceased            bool
current_status    object
branch_id          Int64
branch_name       object
college           object
gender            object
location          object
referrer           Int64
league            object
username          object
dtype: object

In [11]:
def init_profile(id):
    url = constants['LLHEADER'] + f'/profiles.php?{str(id)}'
    main_data = sess.get(url)
    html_text = main_data.text
    soup = BeautifulSoup(html_text, 'html.parser')
    ret = dict()

    if 'This player is inactive.' in soup.text:
        ret['status'] = 'inactive'
        ret['name'] = soup.find('div', attrs={'class': 'namediv'}).text.strip()
    else:
        box = soup.find('div', attrs={'class': 'topcont'})
        if '\t\tPassed away' in box.text:
            ret['status'] = 'deceased'
        else:
            ret['status'] = box['class'][-1]
        ret['branch'] = int(soup.findAll('div', attrs={'class': 'demog_row'})[1].find('a')['href'].split('?')[-1])
        genloc = soup.findAll('div', attrs={'class': 'demog_row'})[0].text.strip().split('\n\n')
        ret['gender'] = genloc[0].strip().split(': ')[-1]
        ret['location'] = genloc[1].strip().split(': ')[-1]
        ret['referrer'] = int(soup.findAll('div', attrs={'class': 'demog'})[4].find('a')['href'].split('?')[-1])
        ret['name'] = soup.find('h1', attrs={'class': 'namecss'}).text
        ret['league'] = soup.find('div', attrs={'class': 'leaguelogodiv'}).text.strip()
        record_rows = soup.find('table', attrs={'summary': 'Data table for career record'}).findAll('tr')[1:]
        record_dic = dict()
        for row in record_rows:
            level, record, _ = row.text.strip().split('\n')
            level = level.split(' ')[-1]
            record_dic[level] = {
                'W': int(record.split('-')[0]),
                'L': int(record.split('-')[1]),
                'T': int(record.split('-')[2]),
            }
        ret['record'] = pd.DataFrame.from_dict(record_dic, orient='index')
        data_table = soup.find('table', attrs={'class': 'std sortable this_sea std_bord'})
        rows = data_table.findAll('tr')
        headers = ['Category', 'Career', 'Career Pct', 'Lg Pct', 'Recent Szn', 'Szn Pct', 'Szn Lg Pct', 'Rundle Pct']
        tbl = [[int_or_float_or_str(x.text) for x in row.findAll('td')] for row in rows[1:]]
        df = pd.DataFrame(tbl, columns=headers)
        df['Career TCA'] = df.apply(lambda row: int(row['Career'].split('-')[0]), axis=1)
        df['Career Q'] = df.apply(lambda row: int(row['Career'].split('-')[1]), axis=1)
        print(df)
        df['Szn TCA'] = df.apply(lambda row: int(row['Recent Szn'].split('-')[0]), axis=1)
        df['Szn Q'] = df.apply(lambda row: int(row['Recent Szn'].split('-')[1]), axis=1)
        df.drop(['Career', 'Career Pct', 'Recent Szn', 'Szn Pct'], axis=1, inplace=True)
        new_headers = ['Category', 'Career TCA', 'Career Q', 'Lg Pct', 'Szn TCA', 'Szn Q', 'Szn Lg Pct', 'Rundle Pct']
        df = df[new_headers]
        ret['Q_data'] = df
    return ret
init_profile(23168)

       Category    Career  Career Pct  Lg Pct Recent Szn Szn Pct Szn Lg Pct  \
0          MATH     45-60       0.750   0.479          -                      
1     GEOGRAPHY   119-221       0.538   0.489        1-2     0.5      0.514   
2   CURR EVENTS     38-80       0.475   0.547        0-1     0.0      0.407   
3   GAMES/SPORT    98-226       0.434   0.491        2-2     1.0       0.55   
4     AMER HIST    54-127       0.425   0.554        1-1     1.0      0.693   
5      LANGUAGE     38-92       0.413   0.516        0-1     0.0      0.888   
6       SCIENCE   119-299       0.398   0.495        2-3   0.667      0.481   
7    WORLD HIST    92-233       0.395   0.480        2-2     1.0      0.398   
8     LIFESTYLE    44-120       0.367   0.526        0-1     0.0      0.606   
9      BUS/ECON     31-96       0.323   0.538          -                      
10          ART    39-137       0.285   0.403        1-1     1.0      0.715   
11   TELEVISION    46-169       0.272   0.530       

ValueError: invalid literal for int() with base 10: ''

In [91]:
def get_past_answer(season, md): # ONLY WORKS FOR YOU!
    data = {'season': int_or_float_or_str(season), 'matchday': int(md)}
    res = sess.post('https://www.learnedleague.com/thorsten/pastanswers.php', data=data)
    soup = BeautifulSoup(res.text, 'html.parser')
    table = soup.find('table', attrs={'class': 'qtable'})
    rows = table.findAll('tr')
    headers = rows[0].text.strip().split('\n')
    datapts = []
    for idx, row in enumerate(rows[1:]):
        ret = [x.text for x in row.findAll('td')]
        try:
            ret[-1] = 1 if row.find('img')['src'].split('/')[-1] == 'submitted.png' else 0
        except:
            ret[-1] = -1
        ret[:0] = [int_or_float_or_str(season), md, idx+1]
        datapts.append(ret)
    return datapts

[[92,
  11,
  1,
  '1. An Isaac Bashevis Singer short story subtitled "The Yeshiva Boy" was turned into what 1983 movie musical (the directorial debut of the film\'s star)?',
  'YENTL',
  'little drummer boy',
  0],
 [92,
  11,
  2,
  '2. What is the name in biochemistry for the pyrimidine base, part of the genetic code of RNA, that pairs with adenine during the process of transcription and is replaced by thymine in DNA?',
  'URACIL',
  'cytosine',
  0],
 [92,
  11,
  3,
  '3. Among the dumb challenges that went viral on TikTok and other social media in 2021, one of the more dangerous emerged in August, in which a brave (or foolish) soul would attempt to traverse an arrangement of what precariously stacked objects, with the clips often ending in group laughter and personal injury?',
  'MILK CRATES',
  'milk crates',
  1],
 [92,
  11,
  4,
  '4. What colloquial term for a metaphorical dessert, the eating of which means to apologize or be submissive to the point of humiliation, dates fro

In [97]:
def all_past_answers():
    past_answer_page = sess.get(constants['LLHEADER'] + '/thorsten/pastanswers.php')
    soup = BeautifulSoup(past_answer_page.text, 'html.parser')
    season_opts = [int_or_float_or_str(opt['value']) for opt in soup.find('form').find('div').findAll('option')]
    past_answers = []

    for season in season_opts:
        length = 25 if isinstance(season, int) else 11
        for md in range(1, length + 1):
            past_answers.extend(get_past_answer(season, md))
    return past_answers

In [96]:
# columns = ['season', 'md', 'qnum', 'question', 'answer', 'attempt', 'ruling']
# pa_df = pd.DataFrame(past_answers, columns=columns)
# pa_df['attempt'] = pa_df['attempt'].str.upper()
# pa_df.to_csv('./personal_data/my_answers.csv')

In [108]:
def get_q_history(id):
    global sess
    main_data = sess.get(f'https://www.learnedleague.com/profiles.php?{(str(id))}&9')
    html_text = main_data.text
    soup = BeautifulSoup(html_text, 'html.parser')
    if 'In order to view LL player profile information, you must be an active LearnedLeague member, and logged in.' in soup.text:
        sess = get_session()
        main_data = sess.get(f'https://www.learnedleague.com/profiles.php?{(str(id))}&9')
        html_text = main_data.text
        soup = BeautifulSoup(html_text, 'html.parser')
    if 'This player is inactive.' in soup.text:
        return []
    q_by_cat = soup.find('div', attrs={'class': 'qhistory'}).find_all('ul', attrs={'class': 'mktree'})
    ph = []
    for qcat in q_by_cat:
        qs = qcat.find('table', attrs={'class': 'qh'}).find_all('tr')[1:]
        for q in qs:
            data = q.find_all('td')
            season, md, qnum = data[0].text.split()
            season, md, qnum = int_or_float_or_str(season[2:]), int_or_float_or_str(md[2:]), int_or_float_or_str(qnum[1:])
            # qid = '-'.join([season, md, qnum])
            correct = data[2]['class'][1] == 'g'
            ph.append([season, md, qnum, int(id), correct])
    return ph
get_q_history(23168)

[[93, 19, 5, 23168, True],
 [93, 16, 3, 23168, True],
 [93, 15, 2, 23168, True],
 [93, 14, 5, 23168, True],
 [93, 8, 4, 23168, False],
 [93, 2, 5, 23168, False],
 [92, 24, 6, 23168, True],
 [92, 18, 4, 23168, True],
 [92, 15, 2, 23168, False],
 [92, 12, 1, 23168, True],
 [92, 7, 2, 23168, False],
 [92, 3, 5, 23168, True],
 [91, 23, 5, 23168, False],
 [91, 21, 2, 23168, False],
 [91, 18, 4, 23168, False],
 [91, 16, 4, 23168, True],
 [91, 13, 6, 23168, False],
 [91, 11, 5, 23168, True],
 [91, 7, 2, 23168, True],
 [91, 2, 6, 23168, False],
 [90, 23, 4, 23168, True],
 [90, 17, 1, 23168, True],
 [90, 15, 1, 23168, True],
 [90, 13, 6, 23168, False],
 [90, 11, 2, 23168, True],
 [90, 3, 3, 23168, True],
 [89, 23, 5, 23168, False],
 [89, 22, 1, 23168, True],
 [89, 13, 5, 23168, False],
 [89, 11, 6, 23168, False],
 [89, 9, 5, 23168, False],
 [89, 8, 1, 23168, False],
 [89, 2, 5, 23168, False],
 [88, 23, 6, 23168, False],
 [88, 20, 1, 23168, False],
 [88, 18, 4, 23168, True],
 [88, 16, 6, 23168, 

In [109]:
import time
with open('player_info.json', 'r') as fp:
    player_info = json.load(fp)
player_q_history = dict()
st = time.time()
player_q_history = []
for idx, key in enumerate(list(player_info.keys())):
    try: player_q_history.extend(get_q_history(key))
    except: continue
    if idx % 1000 == 0:
        print(f'done {idx}')
print(f'total time: {time.time() - st}')
len(player_q_history)

done 0
done 1000
done 2000
done 3000
done 4000
done 5000
done 6000
done 7000
done 8000
done 9000
done 10000
done 11000
done 12000
done 13000
done 14000
done 15000
done 16000
done 17000
done 18000
done 19000
done 20000
done 21000
done 22000
done 23000
done 24000
done 25000
done 26000
done 27000
total time: 42470.00387620926


37244514

In [121]:
pqh_df = pd.DataFrame(player_q_history, columns=['season', 'md', 'qnum', 'id', 'correct'])
pqh_df.correct = pqh_df.apply(lambda row: 1 if row['correct'] == True else 0, axis=1)
pqh_df.head()

,season,md,qnum,id,correct
0,93,19,5,39204,0
1,93,16,3,39204,0
2,93,15,2,39204,1
3,93,14,5,39204,1
4,93,8,4,39204,0


In [122]:
pqh_df.to_csv('./player_question_history.csv', index=False)

In [125]:
pqh_df.shape

(37244514, 5)

In [52]:
def get_personal_matches(id: int):
    url = constants['LLHEADER'] + f'/profiles.php?{str(id)}&7'
    main_data = sess.get(url)
    html_text = main_data.text
    soup = BeautifulSoup(html_text, 'html.parser')
    past_seasons = soup.find('div', attrs={'class': 'pastseasons'}).find_all('div', {'class': 'fl_latest fl_l_l'})
    # print(int(past_seasons[0].find('h2').find('a').text[2:]))
    rundle_participation = []
    personal_matches = []
    for season in past_seasons:
        seas = int(season.find('h2').find('a').text[2:])
        rundle = season.find('h3').find('a')['href'].split('&')[1].split('_')
        div = 0
        if len(rundle) > 2:
            lvl, league, div = rundle[0], rundle[1], int(rundle[3])
        elif len(rundle) == 1 and rundle[0] == 'Championship': # then in the old rundle Championship.
            lvl = 'A'
            league = 'Championship' # assigning level to A since that is closest.
        else:
            lvl, league = rundle
        # print(seas, lvl, league, div)
        matches = season.find('tbody').find_all('tr')
        rundle_participation.append([id, seas, lvl, league, div])
        for match in matches:
            md, opp, outcome, score, record, rank = match.find_all('td')
            # print(md, opp, outcome, score, record, rank)
            md = md.text.split()[2]
            opp = int(opp.find('a')['href'].split('?')[1])
            outcome = outcome.text.upper()
            personal_match_id = int(score.find('a')['href'].split('=')[1])
            in_season_rec = list(map(lambda x: int(x), record.text.split('-')))
            rank = int(rank.text)
            # print(md, opp, outcome, personal_match_id, in_season_rec, rank)
            personal_matches.append([personal_match_id, id, outcome])
    return personal_matches, rundle_participation
get_personal_matches(23168)
# with open('player_info.json', 'r') as fp:
#     player_info = json.load(fp)
# player_info

AttributeError: 'NoneType' object has no attribute 'find_all'